So, firstly, I need to import the necesssary packages. load env variables. load openAI chat model.


Here, I will combine, Outfit attributes, skin tone, wether api data. 
and generate response based on the data using langChain-> Model, prompt, parsers, Chains and Memory

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv, find_dotenv

load_dotenv(dotenv_path=r"C:\Users\hasan\Rafi_SAA\python_practices\LLM_Langchain\llm_openai\.env")


True

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableSequence
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

In [3]:
chat = ChatOpenAI(
    model="openai/gpt-4o",
    openai_api_key=os.getenv("GPT_github_access_token"),
    openai_api_base="https://models.github.ai/inference",
    temperature = 0.5,
    frequency_penalty = 0.2,
    presence_penalty = 0.2

)

C:\Users\hasan\AppData\Local\Temp\ipykernel_21396\1285999360.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(
WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.


Taking weathear API data and filering for specific data fields.

In [6]:
import os
import requests
from dotenv import load_dotenv

# Load environment variables
load_dotenv(dotenv_path=r"C:\Users\hasan\Rafi_SAA\python_practices\LLM_Langchain\llm_openai\.env")
API_KEY = os.getenv("weather_api_key")
BASE_URL = "https://api.weatherapi.com/v1"



Current weather data for a specific city

In [ ]:
def get_current_weather(city: str) -> str:
    """
    Fetch current weather for a given city.
    Returns a human-readable summary.
    """
    url = f"{BASE_URL}/current.json?key={API_KEY}&q={city}&aqi=no"
    response = requests.get(url)
    data = response.json()
    
    current = data.get("current", {})
    condition = current.get("condition", {}).get("text", "Unknown")
    temp_c = current.get("temp_c", "Unknown")
    humidity = current.get("humidity", "Unknown")
    wind_kph = current.get("wind_kph", "Unknown")
    
    
    return f"Current weather in {city}: {condition.lower()}, temperature around {temp_c}°C, humidity {humidity}%, wind speed {wind_kph} kph."


In [8]:
get_current_weather("barcelona")

'Current weather in barcelona: clear, temperature around 14.1°C, humidity 51%, wind 16.6 kph.'

Forecast data-- for a specific city.

In [26]:
from dateparser.search import search_dates
import requests
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=r"C:\Users\hasan\Rafi_SAA\python_practices\LLM_Langchain\llm_openai\.env")
API_KEY = os.getenv("weather_api_key")
BASE_URL = "https://api.weatherapi.com/v1"

def get_forecast_weather(city: str, user_input: str) -> str:
    # Extract date from input using search_dates
    dates = search_dates(user_input, settings={'PREFER_DATES_FROM': 'future'})
    if not dates:
        return "Could not determine the date. Please specify a valid date."
    
    date_obj = dates[0][1]
    date_str = date_obj.strftime("%Y-%m-%d")
    
    # Fetch forecast
    url = f"{BASE_URL}/forecast.json?key={API_KEY}&q={city}&dt={date_str}&aqi=no&alerts=no"
    data = requests.get(url).json()
    
    forecast_day = data.get("forecast", {}).get("forecastday", [{}])[0].get("day", {})
    condition = forecast_day.get("condition", {}).get("text", "Unknown")
    avg_temp = forecast_day.get("avgtemp_c", "Unknown")
    max_temp = forecast_day.get("maxtemp_c", "Unknown")
    min_temp = forecast_day.get("mintemp_c", "Unknown")
    chance_of_rain = forecast_day.get("daily_chance_of_rain", "Unknown")
    wind_kph = forecast_day.get("maxwind_kph", "Unknown")

    
    return (
        f"Forecast for {city} on {date_str}: {condition.lower()}, "
        f"avg temp around {avg_temp}°C (min {min_temp}°C / max {max_temp}°C), "
        f"chance of rain {chance_of_rain}%."
        f"Max wind speed of the day {wind_kph} kph"
    )



In [28]:
print(get_forecast_weather("New York", " on 28th October"))
get_current_weather("new York")

Forecast for New York on 2025-10-28: partly cloudy , avg temp around 7.9°C (min 5.3°C / max 11.4°C), chance of rain 0%.Max wind speed of the day 15.5 kph


'Current weather in new York: clear, temperature around 9.4°C, humidity 66%, wind 13.0 kph.'

Taking data from outfit extraction api 

In [33]:
import requests
outfit_api = "http://127.0.0.1:5000/api/extract_outfit"

image_path = r"C:\Users\hasan\Rafi_SAA\Mobile_app_Cvindal1\PROJECT_CVINDAL1\test_img\pink_coat.jpg"

with open(image_path, "rb") as f:
    files = {"image": f}
    response = requests.post(outfit_api, files= files)

if response.status_code == 200:
    data = response.json()
    # print("API output", data)
    print(data)

else:
    print("error", response.status_code, response.text)



{'attributes': {'additional_details': 'lapel collar, button closure, pockets', 'clothing_type': 'coat', 'color': 'pink', 'pattern': 'solid', 'sleeve_length': 'long', 'style': 'casual'}, 'success': True}


--------------------------------------------------------------------------------------

Now utilizing LangCHain with memory, passing all the data into my gpt-4o model. \


=================================================

In [37]:
from langchain.chains import ConversationChain
memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=chat,
    memory = memory,
    verbose = True,
)

C:\Users\hasan\AppData\Local\Temp\ipykernel_5404\2629504581.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
C:\Users\hasan\AppData\Local\Temp\ipykernel_5404\2629504581.py:4: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(


In [38]:
skin_tone ="mid-light"
weather_str = get_current_weather("New York")

prompt = f"""
You are a fashion assistant. Based on the information below, generate a personalized outfit recommendation for the user.

1. Outfit attributes (from the uploaded image):
{data}

2. Skin tone:
{skin_tone}

3. Current or forecast weather:
{weather_str}

Instructions:
- Suggest a complete outfit suitable for the weather.
- Include clothing type, colors, and patterns if relevant.
- Make it personalized for the user's skin tone.
- Keep the recommendation practical and stylish.
- Use concise, natural language.

Provide your answer in **one paragraph**, as if you are giving advice to the user directly.
"""

In [39]:
response = conversation.run(prompt)

C:\Users\hasan\AppData\Local\Temp\ipykernel_5404\4267660032.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = conversation.run(prompt)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 
You are a fashion assistant. Based on the information below, generate a personalized outfit recommendation for the user.

1. Outfit attributes (from the uploaded image):
{'attributes': {'additional_details': 'lapel collar, button closure, pockets', 'clothing_type': 'coat', 'color': 'pink', 'pattern': 'solid', 'sleeve_length': 'long', 'style': 'casual'}, 'success': True}

2. Skin tone:
mid-light

3. Current or forecast weather:
Current weather in New York: clear, temperature around 8.3°C, humidity 68%, wind 13.3 kph.

Instructions:
- Suggest a complete outfit suitable for the weather.
- Include clothing type, colors, and patterns if relevant.
- Make it personal

In [41]:
print(response)

For a crisp New York day at 8.3°C, pair your pink casual coat with a cream-colored turtleneck sweater for warmth and a soft contrast to your mid-light skin tone. Opt for high-waisted dark wash skinny jeans to balance the coat’s solid color and keep the look sleek. Add ankle boots in taupe or light gray for a neutral, stylish touch, and accessorize with a chunky knit scarf in a matching cream tone. To complete the outfit, go for gold or rose gold jewelry to complement the warmth of the pink coat, and carry a structured crossbody bag in a muted beige or blush hue. This ensemble is cozy, practical, and effortlessly chic for the season!


In [40]:
conversation.predict(input= "what based on my given data is that dress good for the weather?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 
You are a fashion assistant. Based on the information below, generate a personalized outfit recommendation for the user.

1. Outfit attributes (from the uploaded image):
{'attributes': {'additional_details': 'lapel collar, button closure, pockets', 'clothing_type': 'coat', 'color': 'pink', 'pattern': 'solid', 'sleeve_length': 'long', 'style': 'casual'}, 'success': True}

2. Skin tone:
mid-light

3. Current or forecast weather:
Current weather in New York: clear, temperature around 8.3°C, humidity 68%, wind 13.3 kph.

Instructions:
- Suggest a complete outfit suitable for the weather.
- Include clothing type, colors, and patterns if relevant.
- Make it personali

'Yes, the outfit I suggested is well-suited for the current weather in New York, which is clear with a temperature of around 8.3°C (47°F). The pink coat with long sleeves will provide warmth while being lightweight enough for casual wear. The cream turtleneck sweater adds an extra layer of insulation to keep you cozy, and pairing it with high-waisted jeans ensures comfort and coverage against the chilly breeze. The ankle boots are practical for walking in cool weather, and the chunky knit scarf not only adds style but also protects your neck from the cold. Overall, the outfit balances warmth and functionality without compromising on style, making it ideal for this crisp fall day!'